In [15]:
import pandas as pd 
import numpy as np
import json
import pandas_market_calendars as mcal

# Minutely Data

## Configuration

In [ ]:
# List off functions for modifying the return

## Import IQFeed Stocks

1. Sort news by stock ticker
2. For each stock ticker do: 
3. Download prices for stock ticker
4. 

In [3]:
ticker = "FITB"

In [126]:
df = pd.read_parquet("data/unraw2_bzg/data-5.parquet")
prices = pd.read_parquet(f"D:/IQFeedData/{ticker}_1min.parquet")

In [127]:
nyse_cal = mcal.get_calendar('NYSE')

In [128]:
def get_appropriate_closing_time(time: pd.Timestamp) -> pd.Timestamp:
    if (time.hour < 9) or ((time.hour == 9) and (time.minute < 30)):
        return pd.Timestamp(year=time.year, month=time.month, day=time.day, hour=16, minute=0, tz=time.tz)
    else:
        valid_days = [x.date() for x in nyse_cal.valid_days(start_date=time.date(), end_date=time.date() + pd.DateOffset(days=10))]
        i = 1
        while True:
            new_time = time + pd.DateOffset(days=i)
            if new_time.date() in valid_days:
                return pd.Timestamp(year=new_time.year, month=new_time.month, day=new_time.day, hour=16, minute=0, tz=time.tz)
            if i == 7:
                return ValueError()
            i += 1

In [129]:
ts = df.time.dt.ceil("min").iloc[5]
print(ts)
new_ts = get_appropriate_closing_time(ts)
print(new_ts)

2011-04-08 07:00:00-04:00
2011-04-08 16:00:00-04:00


In [130]:
# TODO: This can be *improved* by saying that if we are very close to completing the minute e.g. :55, 
# then we dont take the next candle (T+1), but the candle after the next(T+2).
df.loc[:, "entry_time"] = df.loc[:, "time"].dt.ceil("min")
df.loc[:, "nn_exit_time"] = df.loc[:, "time"].apply(get_appropriate_closing_time)

In [140]:
df.head()

,time,stocks,author,title,channels,body,entry_time,nn_exit_time
__null_dask_index__,,,,,,,,
988369,2011-04-07 22:14:05-04:00,ETF,Benzinga,Passive Management Gains Popularity amongst In...,[],(via COMTEX News Network)-- The Exchange Trad...,2011-04-07 22:15:00-04:00,2011-04-08 16:00:00-04:00
988370,2011-04-07 22:14:22-04:00,ETF,Benzinga,Passive Management Gains Popularity amongst In...,[],(via COMTEX News Network)-- The Exchange Trad...,2011-04-07 22:15:00-04:00,2011-04-08 16:00:00-04:00
988380,2011-04-07 23:02:04-04:00,SOL,Benzinga,ReneSola Ltd Announces Exercise and Closing of...,[],"JIASHAN, China, April 7, 2011 /PRNewswire-Asia...",2011-04-07 23:03:00-04:00,2011-04-08 16:00:00-04:00
988382,2011-04-07 23:08:04-04:00,DNP,Benzinga,DNP Select Income Fund Inc. Section 19(a) Notice,[],"CHICAGO, April 7, 2011 /PRNewswire/ -- ...",2011-04-07 23:09:00-04:00,2011-04-08 16:00:00-04:00
988569,2011-04-08 07:00:00-04:00,AMD,Benzinga,"WILD Flavors, Inc. Celebrates Opening Day in C...",[],"# WILD Flavors, Inc. Celebrates Opening Day in...",2011-04-08 07:00:00-04:00,2011-04-08 16:00:00-04:00


In [141]:
ticker_news = df[df.stocks == ticker]

In [134]:
import pytz
eastern = pytz.timezone('US/Eastern')
prices.loc[:, "time"] = prices.loc[:, "time"].dt.tz_localize(None)
prices.loc[:, "time"] = prices.loc[:, "time"].dt.tz_localize(eastern)

In [150]:
ticker_news.iloc[0]

time                                    2011-04-14 10:40:05-04:00
stocks                                                       FITB
author                                                   Benzinga
title           Shares of FITB Down 3.2% Since Downtrend Call ...
channels                                       ["Press Releases"]
body            (via COMTEX News Network)--\n\nSmarTrend ident...
entry_time                              2011-04-14 10:41:00-04:00
nn_exit_time                            2011-04-15 16:00:00-04:00
Name: 1004285, dtype: object

In [149]:
print(prices.loc[prices.time == ticker_news.iloc[0].entry_time])
print("")
print(prices.loc[prices.time == ticker_news.iloc[0].nn_exit_time])

                            time   open    high    low   close   volume
135242 2011-04-14 10:41:00-04:00  13.55  13.555  13.55  13.555  10589.0

                            time  open  high    low  close    volume
135966 2011-04-15 16:00:00-04:00  13.8  13.8  13.79  13.79  142594.0


# Daily Data

## Import Stocks

In [ ]:
stocks = pd.read_pickle("data/stocks.pkl")

In [ ]:
stocks.head(3)

In [ ]:
stock_tickers = set(stocks.index.get_level_values("ID").unique())
len(stock_tickers)
#test

## Import Stories

In [19]:
#stories = pd.read_pickle("data/stories.pkl")
stories = pd.read_parquet("data/raw_bzg/story_df_raw_2019.parquet")

In [ ]:
stories_tickers = set(stories.stocks.unique())

## Parse stocks

In [ ]:
assert stories.stocks.dtype == stocks.index.dtypes[1]

In [ ]:
def add_targets(df):
    required_columns = ["Close", "High", "Low", "Open"]
    # df.loc[:, "IntradayReturn"] = df["Close"]/df["Open"] - 1
    df.loc[:, "CloseToCloseReturn"] = df["Close"] / df.shift(1)["Close"] - 1
    # df.loc[:, "NextDayReturn"] = df.shift(-1)["Close"] / df.shift(-1)["Open"] - 1
    # df.loc[:, "CloseToNextOpen"] = df.shift(-1)["Open"] / df["Close"] - 1
    return df

In [ ]:
stocks.index.dtypes

In [ ]:
stocks.loc[:, ["IntradayReturn", "NextDayReturn"]] = np.nan
stocks = stocks.swaplevel(0, 1).sort_index(ascending=[True, True])

In [ ]:
stocks = stocks.groupby("ID", as_index=False).apply(add_targets)
stocks.index = stocks.index.droplevel(None)

## Parse Stories

In [ ]:
from pandas.tseries.offsets import BDay

In [ ]:
# PARAMETER 
typ = "CloseToCloseReturn"

In [ ]:
def get_appropriate_date(timestamp, typ):
    if typ == "CloseToCloseReturn":
        # TODO: Some noise here due to closing auction?
        if timestamp.hour < 16: return timestamp.date()
        if timestamp.hour >= 16: return timestamp.date() +  BDay(1)

In [ ]:
# test
get_appropriate_date(stories.NewsTimestamp.iloc[4], typ="CloseToCloseReturn")

### Date assignment

In [ ]:
# If we use Intraday return then news should only be between 9:40 am and 4pm (us trading hours).
# If we use close-to-close return then news for this days CTC should be between yesterday 4pm and today 4pm. 
stories.loc[:, "Date"] = stories.NewsTimestamp.apply(lambda x: get_appropriate_date(x, typ))
stories = stories.astype({"Date":'datetime64[ns]'})

## Merging

In [ ]:
stories.rename(columns=dict(stocks="ID"), inplace=True)

In [ ]:
stories.columns

In [ ]:
stocks.columns

In [ ]:
dataset = stories[["Date", "NewsTimestamp", "ID", "body"]].\
    merge(stocks[
        [
            # "IntradayReturn", 
            # "NextDayReturn", 
            "CloseToCloseReturn"]
         ], on=["Date", "ID"], how="inner")

In [ ]:
dataset[dataset.isna().sum(axis=1) > 0]

In [ ]:
dataset = dataset.dropna()

	Date	NewsTimestamp	ID	body	CloseToCloseReturn

In [ ]:
dataset.to_pickle("data/dataset.pkl")

## Create train-test-split 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import pickle

In [ ]:
dataset = pd.read_pickle("data/dataset.pkl")
assert dataset.index.is_unique

In [ ]:
test_size = 0.2
seed = 420
### Train-test split -> Auslagern
train_idx, test_idx = train_test_split(dataset.index, test_size=0.2, random_state=seed)

In [ ]:
test_idx

## Filter training set for general stock market events

In [ ]:
# Select S&P
SnP = stocks.query("ID == 'A0AET0'")

In [ ]:
SnP = add_targets(SnP)

In [ ]:
alpha = 0.1 # Percentage observations classified as too extreme to be used in the training set  

In [ ]:
target_col = "CloseToCloseReturn"

In [ ]:
lower_quantile = SnP.loc[:, target_col].quantile(alpha/2)

In [ ]:
upper_quantile = SnP.loc[:, target_col].quantile(1-alpha/2)

In [ ]:
print(f"Upper Quantile: {upper_quantile:.4f}. Lower Quantile: {lower_quantile:.4f}")

In [ ]:
mask = (SnP.loc[:, target_col] >= lower_quantile) & (SnP.loc[:, target_col] <= upper_quantile)

# Only select dates where SnP behaved calmly
allowed_dates = SnP.loc[mask, :].index.get_level_values("Date")

In [ ]:
# Now trim  training set
train_dat = dataset.loc[train_idx, :]
adj_train_idx = train_dat.loc[train_dat.Date.isin(allowed_dates)].index

In [ ]:
train_idx

In [ ]:
adj_train_idx

## Save train and test indices

In [ ]:
with open('data/dataset_train_test_idx.pkl', 'wb') as f:
    pickle.dump((adj_train_idx, test_idx), f)